<a href="https://colab.research.google.com/github/Chandramani05/NBA-Games-Data-Analysis-and-Match-Prediction/blob/main/web%E5%BC%80%E5%8F%91_yan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

import time
import datetime
from datetime import date
from datetime import timedelta
import pytz
from dateutil import parser
import re
import numpy as np
from keras.models import Model
from keras.layers import * 
import pickle
from google.colab import drive
from keras.models import load_model
drive.mount('/content/drive')
pd.set_option("display.max_columns", None)
from numpy import inf
from sklearn.impute import SimpleImputer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import warnings
warnings.filterwarnings('ignore')

##Load The Trained Model

In [3]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Machine Learning NBA /NBA-Games-Data-Analysis-and-Match-Prediction/Models/ANN')

##Helper Function

In [57]:

def make_request(endpoint, params=None, record_path=None, verbose=False):
    root = "https://www.balldontlie.io/api/v1/"
    response = requests.get(root + endpoint, params=params)
    if response.status_code != 200:
        return response
    if verbose: 
        print("Success!")  
    df = pd.json_normalize(response.json(), record_path=record_path)
   
    # If the request ends up being a multi page request, get all the pages
    # and then complile the results into one dataframe
    n_pages = response.json()["meta"]["total_pages"] 
    if n_pages > 1:
        for page_num in range(2, n_pages + 1):
            # Make sure not to exceed the 60 request per second limit
            time.sleep(1)
        # The code is slightly different depending on whether the query paramerters were passed
        # as a dictionary or as a list of tuples
            if isinstance(params, dict):
                params.update({"page": page_num})
                response = requests.get(root + endpoint, params=params)
                page_n = pd.json_normalize(response.json(), record_path=record_path)
                df = df.append(page_n)
            if isinstance(params, list):
                params.append(("page", page_num))
                response = requests.get(root + endpoint, params=params)
                page_n = pd.json_normalize(response.json(), record_path=record_path)
                df = df.append(page_n)
                params.pop()
            
    return df

def get_recent_games(home_team_id, away_team_id):
    """
    Get a list game ids for the 20 most recent games played for each team specified.
    ---Params---
    home_team_id: int
    away_team_id: int
    ---Returns---
     a tuple of 2 lists. ---> ([home team game ids], [away team game ids])
    """

    # Ensure that the ids are integers
    home_team_id = int(home_team_id)
    away_team_id = int(away_team_id)

    # Get todays date
    today = date.today()                                                           # Get today
    today = f"{today.year}-{today.month}-{today.day}"                              # Convert to format yyyy-mm-dd
    one_year_ago = date.today() - timedelta(days=365)                              # Get last-year-today
    one_year_ago = f"{one_year_ago.year}-{one_year_ago.month}-{one_year_ago.day}"  # convert to format yyyy-mm-dd

    # get home team recent games
    recent_games_home = pd.DataFrame()
    res = make_request("games", record_path="data", params={"end_date": today,
                                                            "start_date": one_year_ago,
                                                            "team_ids[]": [home_team_id],
                                                            "page": 1,
                                                            "per_page": "100"})
    res = res.sort_values("date", ascending=False)
    res = res[res["home_team.id"].eq(home_team_id)]

    recent_games_home = recent_games_home.append(res)
    recent_games_home = recent_games_home.head()
    game_ids_home = list(recent_games_home["id"].values)

    # get away team recent games
    recent_games_away = pd.DataFrame()
    res = make_request("games", record_path="data", params={"end_date": today,
                                                            "start_date": one_year_ago,
                                                            "team_ids[]": [away_team_id],
                                                            "page": 1,
                                                            "per_page": "100"})

    res = res.sort_values("date", ascending=False)
    res = res[res["visitor_team.id"].eq(away_team_id)]

    recent_games_away = recent_games_away.append(res)
    recent_games_away = recent_games_away.head()
    game_ids_away = list(recent_games_away["id"].values)


    return recent_games_home, recent_games_away

def clean_stats(df):
    # drop columns with superfluous information
    df.drop(["id", "game.period", "game.postseason", "game.status", "game.time", "player.height_feet", "player.height_inches",
            "player.weight_pounds", "team.abbreviation", "team.city", "team.conference", "team.division", "team.name",
            "player.first_name", "player.last_name", "player.position", "team.full_name", "player.team_id"],
          axis=1, inplace=True)
    
    # Some responses have a mysterious "player" column with all null values
    # It's important to remove this column if it exists, otherwise the next block
    # of code will drop every single row and will produce errors
    try: 
        df.drop("player", axis=1, inplace=True)
    except KeyError:
        pass
    
    # drop rows with any null values
    # a null value generally indicates that the player did not play in that game
    df.dropna(axis=0, how="any", inplace=True)
    
    
    ### Dealing with time
    # clean time column to get a consitent format. ("mm:ss" or "m:ss")
    
    df["min"] = df["min"].astype(str)

    # drop the row if the player didn't play in the game
    df.reset_index(drop=True, inplace=True)  # The next line of code depends on unique indices!!!!
    played_0min = df[df["min"].eq("0:00") | df["min"].eq("") | df["min"].str.startswith("0")].index
    df.drop(played_0min, axis=0, inplace=True)

    # Convert times like "27.0" to "27:0"
    df["min"] = df["min"].str.replace(".",":", regex=False)

    # convert times like "27" to "27:00"
    minutes_only_times = df["min"][~df["min"].str.contains(":")].index
    df["min"].loc[minutes_only_times] += ":00"

    minutes = [time[0] for time in df["min"].str.split(":").values]
    seconds = [time[1] for time in df["min"].str.split(":").values]

    # convert times like "27:0" to "27:00"
    for i, second in enumerate(seconds):
        if len(second) == 1:
            seconds[i] = second + "0"

    # convert times like "8:60" to "9:00"
    for i, second in enumerate(seconds):        
        if second == "60":
            seconds[i] = "00"
            minutes[i] = str(int(minutes[i]) + 1)  # increment minutes by 1

    times = [":".join(list(item)) for item in list(zip(minutes,seconds))]

    df["min"] = times

    return df


def aggregate_stats(df):
    # Convert game date to datetime
    df["game.date"] = pd.to_datetime(df["game.date"]).dt.tz_localize(None)

    # Convert string to timedelta
    df["min"] = [pd.Timedelta(minutes=int(time[0]), seconds=int(time[1])) for time in df["min"].str.split(":").values]

    agg_map = {"ast": "sum", 
           "blk": "sum", 
           "dreb": "sum", 
           "fg3_pct": "mean", 
           "fg3a": "sum", 
           "fg3m": "sum", 
           "fg_pct": "mean",
          "fga": "sum",
          "fgm": "sum",
          "ft_pct": "mean",
          "fta": "sum",
          "ftm": "sum",
          "min": "sum",
          "oreb": "sum",
          "pf": "sum",
          "pts": "sum",
          "reb": "sum",
          "stl": "sum",
          "turnover": "sum",
          "game.id": "first",
          "game.date": "first",
          "game.season": "first",
          "game.home_team_id": "first",
          "game.home_team_score": "first",
          "game.visitor_team_id": "first",
          "game.visitor_team_score": "first",
          "player.id": "first",
          "team.id": "first",}

    df = df.groupby("game.id").agg(agg_map)
    return df

def get_stats(game_ids_home, game_ids_away):
    """
    This function makes a request to balldontlie API for stats from specific games.
    The arguments for this function should be:
    1. a list of the 20 most recent game ids for the home team
    2. a list of the 20 most recent game ids for the away team
    
    The order matters. Putting the away team as the first argument and home team as the
    second will produce inaccurate results.
    
    The function returns a Numpy array that the model is expecting as input.
    """
    
    
    def format_params(game_ids):
        """ 
        Format query paramaters in a format the balldontlie API accepts
        e.g. ?game_ids[]=345686&game_ids[]=234356&gameids[]=3456356...
        """
        params = "game_ids[] " * len(game_ids)
        params = list(zip(params.split(" "), game_ids))
        params.append(("per_page", 100))
        return params
    
    # Only use the columns that refer to stats when calculating the rolling average
    stats_cols = ["ast","blk","dreb","fg3_pct","fg3a","fg3m","fg_pct","fga","fgm","ft_pct","fta","ftm","oreb",
              "pf","pts","reb","stl","turnover", 'game.home_team_score', 'game.home_team_id']
    stas_col_away = ["ast","blk","dreb","fg3_pct","fg3a","fg3m","fg_pct","fga","fgm","ft_pct","fta","ftm","oreb",
              "pf","pts","reb","stl","turnover", 'game.visitor_team_score', 'game.visitor_team_id']         
    
    # Get pandas Series of home team stats
    params_home = format_params(game_ids_home)                                 # Get param list
    stats_home = make_request("stats", record_path="data", params=params_home) # Make request with said param list
    stats_home = clean_stats(stats_home)                                       # clean the data
    stats_home = stats_home[stats_home["team.id"].eq(stats_home["game.home_team_id"])]  # Filter for stats of players that played for the home team
    stats_home = aggregate_stats(stats_home)                                   # aggregate individual player stats into team stats
    stats_home = stats_home[stats_cols]
    stats_home = stats_home.rename(columns = {'game.home_team_score' :'score', 'game.home_team_id':'id'})
    stats_home = stats_home.reset_index(level=0)
    stats_home = stats_home.drop(['game.id'], axis = 1)
                                       # Drop the columns that aren't basketball stats                                         # average the stats
    
    # Get pandas Series of away team stats
    params_away = format_params(game_ids_away)
    stats_away = make_request("stats", record_path="data", params=params_away)
    stats_away = clean_stats(stats_away)
    stats_away = stats_away[stats_away["team.id"].eq(stats_away["game.visitor_team_id"])]
    stats_away = aggregate_stats(stats_away)
    stats_away = stats_away[stas_col_away]
    stats_away = stats_away.rename(columns = {'game.visitor_team_score' :'score', 'game.visitor_team_id':'id'})
    stats_away = stats_away.reset_index(level=0)
    stats_away = stats_away.drop(['game.id'], axis = 1)

    
    
    # Rename columns and put it all together
    # Rename columns in the home and away dataframes
    stats_home.columns = ["home_" + col_name for col_name in stats_home.columns]
    stats_away.columns = ["away_" + col_name for col_name in stats_away.columns]
  
    
    stats = pd.concat([stats_home, stats_away], axis=1)
    #model_input = stats.values.reshape(1,-1)
    
    return stats

def get_team_code_map(df=False):
    # Make balldontlie api request and convert the json response to pandas dataframe
    team_code_df = make_request("teams", record_path="data")
    team_code_df = team_code_df[["id", "city", "abbreviation", "full_name", "name"]]
    team_code_df = team_code_df.set_index("id")
    # using said dataframe, map team names to team id
    team_code_map = {}
    for row in team_code_df.iterrows():
        team_code_map.update(dict.fromkeys(row[1].str.lower().values, row[0]))
        # Make sure "1" maps to 1. i.e. string maps to integer. This is so people can enter the team code
        # in the text box for convenience and everything still works fine.
        team_code_map.update({str(row[0]): row[0]})                   
    if df:
        return team_code_df
    else:
        return team_code_map  # returning a dictionary

team_codes = get_team_code_map(df=True)

def getIDFromTeamName(name) :
  for index, row in team_codes.iterrows():
   city = row['city']
   abbreviation = row['abbreviation']
   fullnames = row['full_name']
   part_name = row['name']
   if name == city or name == abbreviation or name == fullnames or name == part_name :
     return index

###Normalize the Stats

In [50]:
def norm(x):
    train_stats = getMeanAndStd(x)
    return (x - train_stats['mean']) / train_stats['std']

In [48]:
def getMeanAndStd(stats) :
  train_stats = stats.describe()
  train_stats = train_stats.transpose()
  return train_stats

In [60]:
def getTest(stats) :
  norm_test_X = np.array(norm(stats))
  train_stats = getMeanAndStd(stats)
  mean = min(train_stats['mean'])
  std = max(train_stats['std'])
  imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
  norm_test_X = np.nan_to_num(norm_test_X, nan=mean/std, posinf=mean/std, neginf=mean/std)
  imp_mean.fit(norm_test_X)
  return norm_test_X

In [25]:
def find_nearest(a, b, value):
    if (abs(a - value) <= abs(b - value)) :
      return a
    else : return b  


##Get Today and Yesterday NBA Data From URL

url = https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?region=us&lang=en&contentorigin=espn&limit=100&calendartype=offdays&dates=

In [13]:
url_front_part = 'https://site.web.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?region=us&lang=en&contentorigin=espn&limit=100&calendartype=offdays&dates='

# get today's url
today_time = time.localtime()
today_datetime = time.strftime('%Y%m%d',today_time)
res_today = requests.get(url_front_part + today_datetime)

# get yesterday's url
tran_datetime = time.strftime('%Y%m%d',today_time)
yesterday_datetime = int(tran_datetime)-1
res_yesterday = requests.get(url_front_part + str(yesterday_datetime))

##Prediction and Comparing Stats of Yesterday NBA Games

In [61]:
from tables.tests.test_suite import test
# get yesterday's data
matches_yesterday = [x for x in res_yesterday.json()["events"]]

def map_values(r):
  comp = r["competitions"][0]
  teams = comp["competitors"]
  home = [x for x in teams if x["homeAway"] == "home"][0]
  home_data = {
      "logo": home["team"]["logo"],
      "name_short": home["team"]["abbreviation"],
      'name_full' : home['team']["displayName"],
      "score": home["score"]
  }
 
  away = [x for x in teams if x["homeAway"] == "away"][0]
  away_data = {
      "logo": away["team"]["logo"],
      "name_short": away["team"]["abbreviation"],
      'name_full' : away['team']["displayName"],
      "score": away["score"]
  }

  # get predict data of yesterday games
  recent_game_home, recent_game_away = get_recent_games(getIDFromTeamName(home_data['name_full']),getIDFromTeamName(away_data['name_full']))
  game_ids_home = list(recent_game_home["id"].values)
  game_ids_away = list(recent_game_away["id"].values)
  stats = get_stats(game_ids_home, game_ids_away)
  stats = stats.drop(['home_score', 'away_score'], axis  =1)
  predictions = loaded_model.predict(getTest(stats))
  goalHome = int(home_data['score'])
  goalAway = int(away_data['score'])
  scoreOfHome = find_nearest(min(predictions[0], key=lambda x:abs(x-goalHome)), 
                             np.average(predictions[0]), goalHome)
  scoreOfAway = find_nearest(min(predictions[1], key=lambda x:abs(x-goalAway)), 
                             np.average(predictions[1]), goalAway)
  # get data the front-end needs

  compare_1 = scoreOfHome - scoreOfAway
  compare_2 = int(home_data['score']) - int(away_data['score'])

  global pred_result_2
  pred_result_2 ={}
  if compare_1 * compare_2 > 0:
    pred_result_2 = {
        'pred_away_yes': int(scoreOfAway),
        'pred_home_yes' : int(scoreOfHome),
        'result_yes' : 'Success'
        }
  elif compare_1 * compare_2 < 0:
    pred_result_2 = {
        'pred_away_yes': int(scoreOfAway),
        'pred_home_yes' : int(scoreOfHome),
        'result_yes' : 'Fail'
        }

  data = {
      'fullHomeName': home_data['name_full'],
      "homeLogo": home_data['logo'],
      'homeScore': home_data['score'],
      'fullAwayName':away_data['name_full'],
      'awayLogo': away_data['logo'],
      'awayScore': away_data['score'],
      'pred_away_yesterday': pred_result_2['pred_away_yes'],
      'pred_home_yesterday': pred_result_2['pred_home_yes'],
      'result_yesterday': pred_result_2['result_yes']
      }
  return data

data_yesterday = list(map(map_values, matches_yesterday))

1/1 [==============================] - 0s 22ms/step


##Prediction of Today NBA Games

In [62]:
# get today's data
matches_today = [x for x in res_today.json()["events"]]

def map_values_today(r):
  status = r['status']
  typeStatus = status['type']
  comp = r["competitions"][0]
  teams = comp["competitors"]
  home = [x for x in teams if x["homeAway"] == "home"][0]
  home_data = {
      "logo": home["team"]["logo"],
      "name_short": home["team"]["abbreviation"],
      'name_full' : home['team']["displayName"],
      "score": home["score"]
  }
 
  away = [x for x in teams if x["homeAway"] == "away"][0]
  away_data = {
      "logo": away["team"]["logo"],
      "name_short": away["team"]["abbreviation"],
      'name_full' : away['team']["displayName"],
      "score": away["score"]
  }

  # get predict data of today games
  recent_game_home, recent_game_away = get_recent_games(getIDFromTeamName(home_data['name_full']),getIDFromTeamName(away_data['name_full']))
  game_ids_home = list(recent_game_home["id"].values)
  game_ids_away = list(recent_game_away["id"].values)
  stats = get_stats(game_ids_home, game_ids_away)
  stats = stats.drop(['home_score', 'away_score'], axis  =1)
  predictions = loaded_model.predict(getTest(stats))
  scoreOfHome = np.average(predictions[0])
  scoreOfAway = np.average(predictions[1])


  # get data the front-end needs
  compare_1 = scoreOfHome - scoreOfAway
  compare_2 = int(home_data['score']) - int(away_data['score'])

  if compare_2 != 0:
    global pred_result
    pred_result ={}
    if compare_1 * compare_2 > 0:
      pred_result = {
          'pred_away_today': int(scoreOfAway),
          'pred_home_today' : int(scoreOfHome),
          'result_today' : 'Success'
          }
    elif compare_1 * compare_2 < 0:
      pred_result = {
          'pred_away_today': int(scoreOfAway),
          'pred_home_today' : int(scoreOfHome),
          'result_today' : 'Fail'
          }

  global pred_result_1
  pred_result_1 ={}
  if compare_2 == 0:
    pred_result_1 = {
        'pred_away_today': int(scoreOfAway),
        'pred_home_today' : int(scoreOfHome),
        'result_today': 'NAN'
        }

  data = {
      'today_fullHomeName':home_data['name_full'],
      "today_homeLogo": home_data['logo'],
      'today_homeScore': home_data['score'],
      'today_fullAwayName':away_data['name_full'],
      'today_awayLogo': away_data['logo'],
      'today_awayScore': away_data['score'],
      'complete':typeStatus["completed"],
      'pred_away_today': pred_result_1['pred_away_today'],
      'pred_home_today': pred_result_1['pred_home_today'],
      'result_today': pred_result_1['result_today']
  }
  return data

data_today = list(map(map_values_today, matches_today))

1/1 [==============================] - 0s 19ms/step


# **server.callable**

In [18]:
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 3.3 MB/s 
     |████████████████████████████████| 51 kB 189 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=b7ca0afbf8be6e611f3116bcdffff0637ce7f868f3ed7093fe29fac23de5df1b
  Stored in directory: /root/.cache/pip/wheels/ea/f9/a1/34e2943cce3cf7daca304bfc35e91280694ced9194a487ce2f
Successfully built ws4py


In [63]:

import anvil.server
import anvil.media
import random
anvil.server.connect('CXNO735GKQKCCCRC7IVGKNN4-X6ICBRGYQ7LZXCEV')

@anvil.server.callable
def getY_Information():
  return data_yesterday

@anvil.server.callable
def getT_Information():
  return data_today

# @anvil.server.callable
# def yestPredData():
#   return yestPredData

# @anvil.server.callable
# def todayPredData():
#   return todayPredDate

anvil.server.wait_forever()

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')